# LBA: Grocery store prices
In this assignment, we model the cost of groceries in different parts of the world. To what extent do
grocery prices vary by country and store brand? Are grocery prices and the geographical
distribution of different grocery stores correlated with other cost-of-living measures- for example,
rent and real estate prices?

## Task 1: Prices
What is the basic average price for each product? You need to think carefully about how to
anchor the basic price for each product since this will depend on the currency used as well
as the distribution of prices.<br><br>

- Anchor in Euros
- Anchor at units that make sense for each category but is base 10 metric, e.g. milk in l, apples and potatoes in kg, meat and butter in 0.1kg. 

In [14]:
## test a currency conversion package
# pip install CurrencyConverter
from currency_converter import CurrencyConverter
from datetime import date
c = CurrencyConverter()
c.convert(100,'USD','EUR', date=date(2020, 9, 24))

85.87376556462

## Task 2: Factors influencing prices
How much does each of the following factors modify the basic price of the product (up or
down)?
- The geographical location (country) of the grocery store.
- Brand of the grocery store. Since we are getting data from multiple countries, you will need to specify whether the store brand is considered budget (cheap), mid-range, or luxury (expensive). This should be based on what you think the general public perception of the store brand is.
- Does price variation by geographical location correlate with variation in rental prices, or
not?

Explain in your report how strong each of these effects is. Which has the greatest influence
on price variation between shops?

Notes:
- Two levels of categorical variables
    - country
    - grocery store rating
- one numeric variable: rent price
- Determine significance and size of the variation between categories